Importar librerias libreriales

In [47]:
import keras
import pandas as pd
from keras.utils.np_utils import to_categorical
import numpy as np
from keras.models import Sequential, Model
from keras.layers import MaxPooling1D, Dropout, Dense, Flatten, Reshape, Input, Concatenate, Activation
from keras.layers import Convolution1D as Conv1D
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
import tensorflow as tf

print('Keras version: ',keras.__version__)

Keras version:  2.3.0


Importar dataset de Bruno

In [48]:
df_X = pd.read_csv("xbsg5x7-cne.csv", header=None)
df_y = pd.read_csv("ybsg5x7-cne.csv", header=None)

y_origen = df_y[0]
y_destino = df_y[1]

X_train, X_test, y_train, y_test = train_test_split(df_X, y_origen, test_size=0.20, random_state=69)

X_test = X_test.to_numpy()
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_train_2 = np.delete(X_train,range(35,50),1)
X_train_reshaped = np.expand_dims(X_train_2, axis=2)
print(X_train.shape,X_train_reshaped.shape, y_train.shape)

(6660, 50) (6660, 35, 1) (6660,)


In [70]:
X_train_reshaped.shape = (6660, 5, 7)
X_train_reshaped[0]


array([[0.24489796, 0.24489796, 0.24489796, 0.24489796, 0.23469388,
        0.13265306, 0.09183673],
       [0.01020408, 0.2244898 , 0.05102041, 0.16326531, 0.20408163,
        0.03061224, 0.04081633],
       [0.07142857, 0.02040816, 0.18367347, 0.1122449 , 0.10204082,
        0.12244898, 0.03061224],
       [0.24489796, 0.24489796, 0.17346939, 0.06122449, 0.15306122,
        0.08163265, 0.14285714],
       [0.24489796, 0.24489796, 0.24489796, 0.24489796, 0.24489796,
        0.21428571, 0.19387755]])

In [69]:
#Aquí se obtinenen los atributos "no vectoriales"
XX_train =X_train[0:,35:50]
XX_train.shape
XX_train[1]

array([ 1.,  1.,  4.,  7.,  2.,  1.,  7.,  4.,  7.,  6., 99., 98., 41.,
        2., 54.])

In [50]:
y_train[:10]

array([1, 1, 4, 0, 3, 0, 1, 3, 0, 4], dtype=int64)

In [6]:
np.unique(y_train)

array([0, 1, 2, 3, 4], dtype=int64)

In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
encoder = OneHotEncoder()
y_train_coded = encoder.fit_transform(y_train.reshape(-1,1)).toarray() 

In [9]:
y_train_coded[:10]

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]])

Modelado de la red

In [21]:
convmodel = Sequential()
convmodel.add(Flatten(input_shape=(7,1)))
convmodel.add(Dense(32, activation='relu'))
convmodel.add(Dense(32, activation='relu'))
#convmodel.add(Conv1D(filters=5, kernel_size=2, activation='relu', input_shape=(7,1)))
#convmodel.add(Conv1D(filters=5, kernel_size=2, activation='relu'))
#convmodel.add(Dropout(0.1))
#convmodel.add(MaxPooling1D(pool_size=2))

In [22]:
#define two different inputs
input1 = Input((7, 1))
input2 = Input((7, 1))
input3 = Input((7, 1))
input4 = Input((7, 1))
input5 = Input((7, 1))
## falta input 4,5 y el 4 -> 6
input6 = Input((4, ))
   
#use the model to get three different outputs (the same model)
out1 = convmodel(input1)
out2 = convmodel(input2)
out3 = convmodel(input3)
out4 = convmodel(input4)
out5 = convmodel(input5)

#concatenate the outputs and add the final part of your model: 
out = Concatenate()([out1,out2,out3,out4,out5])
#out = Flatten()(out)
out = Concatenate()([out,input6])
out = Dense(32, activation='relu')(out)
#convmodel.add(Dropout(0.4))
out = Dense(32, activation='relu')(out)
#convmodel.add(Dropout(0.1))
#out = Dense(500, activation='relu')(out)
out = Dense(5, activation='softmax')(out) 

#create the model taking 2 inputs with one output
model = Model([input1,input2,input3,input4,input5,input6],out)
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy','categorical_crossentropy'])
model.save_weights('model.h5')
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 7, 1)         0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 7, 1)         0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           (None, 7, 1)         0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           (None, 7, 1)         0                                            
____________________________________________________________________________________________

In [18]:
#X_test = X_test.to_numpy()
X_test_2 = np.delete(X_test,range(35,50),1)
X_test_reshaped = np.expand_dims(X_test_2, axis=2)
print(X_test_reshaped.shape)
X_test_reshaped.shape = (1665, 5, 7)
XX_test = X_test[0:,36:40]

X_test1 = X_test_reshaped[0:,0]; X_test1.shape=(1665,7,1)
X_test2 = X_test_reshaped[0:,1]; X_test2.shape=(1665,7,1)
X_test3 = X_test_reshaped[0:,2]; X_test3.shape=(1665,7,1)
X_test4 = X_test_reshaped[0:,3]; X_test4.shape=(1665,7,1)
X_test5 = X_test_reshaped[0:,4]; X_test5.shape=(1665,7,1)

(1665, 35, 1)


In [19]:
m=model.predict([X_test1, X_test2, X_test3, X_test4, X_test5, XX_test])


In [20]:
import sklearn.metrics as skm
#print(m)
m_final = np.argmax(m, axis=1)
print(m_final)
print(skm.accuracy_score(y_test, m_final))
print(skm.f1_score(y_test, m_final, average='macro'))
print(skm.mean_absolute_error(y_test, m_final))

[4 3 3 ... 3 1 3]
0.5195195195195195
0.5173662407369254
1.0228228228228229
